# cjm-transcription-plugin-system

> A flexible plugin system for audio transcription intended to make it easy to add support for multiple backends.

## Install

```bash
$ pip install cjm_transcription_plugin_system
```

## Project Structure

```
nbs/
├── core.ipynb             # Core data structures for audio transcription
├── plugin_interface.ipynb # Abstract base class defining the interface for transcription plugins
└── plugin_manager.ipynb   # Plugin discovery, loading, and lifecycle management system
```

Total: 3 notebooks

## Module Dependencies

```mermaid
graph LR
    core[core<br/>core]
    plugin_interface[plugin_interface<br/>plugin interface]
    plugin_manager[plugin_manager<br/>plugin manager]

    plugin_interface --> core
    plugin_manager --> core
    plugin_manager --> plugin_interface
```

*3 cross-module dependencies detected*

## CLI Reference

No CLI commands found in this project.

## Module Overview

Detailed documentation for each module in the project:

### core (`core.ipynb`)
> Core data structures for audio transcription

#### Import

```python
from cjm_transcription_plugin_system.core import (
    AudioData,
    TranscriptionResult
)
```
#### Classes

```python
@dataclass
class AudioData:
    "Container for audio data and metadata."
    
    samples: np.ndarray  # Audio sample data as a numpy array
    sample_rate: int  # Sample rate in Hz (e.g., 16000, 44100)
    duration: float  # Duration of the audio in seconds
    filepath: Optional[Path]  # Audio file path
    metadata: Dict[str, Any] = field(...)  # Additional metadata
```

```python
@dataclass
class TranscriptionResult:
    "Standardized transcription output."
    
    text: str  # The transcribed text
    confidence: Optional[float]  # Overall confidence score (0.0 to 1.0)
    segments: Optional[List[Dict]] = field(...)  # List of transcription segments with timestamps and text
    metadata: Optional[Dict] = field(...)  # Transcription metadata
```


### plugin interface (`plugin_interface.ipynb`)
> Abstract base class defining the interface for transcription plugins

#### Import

```python
from cjm_transcription_plugin_system.plugin_interface import (
    PluginInterface,
    PluginMeta
)
```
#### Classes

```python
class PluginInterface(ABC):
    "Base interface that all transcription plugins must implement."
    
    def name(
            self
        ) -> str:  # The unique identifier for this plugin
        "Unique plugin identifier."
    
    def version(
            self
        ) -> str:  # The semantic version string (e.g., "1.0.0")
        "Plugin version."
    
    def supported_formats(
            self
        ) -> List[str]:  # List of file extensions this plugin can process
        "List of supported audio formats (e.g., ['wav', 'mp3'])."
    
    def initialize(
            self,
            config: Optional[Dict[str, Any]] = None  # Configuration dictionary for plugin-specific settings
        ) -> None
        "Initialize the plugin with configuration."
    
    def execute(
            self,
            *args,
            **kwargs
        ) -> Any:  # Returns transcription result or plugin-specific output
        "Transcribe audio to text."
    
    def is_available(
            self
        ) -> bool:  # True if all required dependencies are available
        "Check if the plugin's dependencies are available."
    
    def cleanup(
            self
        ) -> None
        "Optional cleanup when plugin is unloaded."
```

```python
@dataclass
class PluginMeta:
    "Metadata about a plugin."
    
    name: str  # The plugin's unique identifier
    version: str  # The plugin's version string
    description: str = ''  # A brief description of the plugin's functionality
    author: str = ''  # The plugin author's name or organization
    package_name: str = ''  # The Python package name containing the plugin
    instance: Optional[PluginInterface]  # The plugin instance
    enabled: bool = True  # Whether the plugin is enabled
```


### plugin manager (`plugin_manager.ipynb`)
> Plugin discovery, loading, and lifecycle management system

#### Import

```python
from cjm_transcription_plugin_system.plugin_manager import (
    PluginManager
)
```
#### Classes

```python
class PluginManager:
    def __init__(self, 
                 plugin_interface: Type[PluginInterface] = PluginInterface,  # The base class/interface plugins must implement
                 entry_point_group: str = "transcription.plugins"  # The entry point group name for plugin discovery
                )
    "Manages plugin discovery, loading, and lifecycle."
    
    def __init__(self,
                     plugin_interface: Type[PluginInterface] = PluginInterface,  # The base class/interface plugins must implement
                     entry_point_group: str = "transcription.plugins"  # The entry point group name for plugin discovery
                    )
        "Initialize the plugin manager."
    
    def discover_plugins(
            self
        ) -> List[PluginMeta]:  # List of discovered plugin metadata objects
        "Discover all installed plugins via entry points.

This method looks for plugins installed as packages that declare
entry points in the specified group."
    
    def load_plugin(
            self,
            plugin_meta: PluginMeta,  # The plugin metadata
            config: Optional[Dict[str, Any]] = None  # Optional configuration for the plugin
        ) -> bool:  # True if successfully loaded, False otherwise
        "Load and initialize a plugin."
    
    def load_plugin_from_module(
            self,
            module_path: str,  # Path to the Python module
            config: Optional[Dict[str, Any]] = None  # Optional configuration for the plugin
        ) -> bool:  # True if successfully loaded, False otherwise
        "Load a plugin directly from a Python module file or package.
Useful for development or local plugins."
    
    def unload_plugin(
            self,
            plugin_name: str  # Name of the plugin to unload
        ) -> bool:  # True if successfully unloaded, False otherwise
        "Unload a plugin and call its cleanup method."
    
    def get_plugin(
            self,
            plugin_name: str  # The name of the plugin to retrieve
        ) -> Optional[PluginInterface]:  # The plugin instance if found, None otherwise
        "Get a loaded plugin instance by name."
    
    def list_plugins(
            self
        ) -> List[PluginMeta]:  # List of metadata for all loaded plugins
        "List all loaded plugins."
    
    def execute_plugin(
            self,
            plugin_name: str,  # Name of the plugin to execute
            *args,  # Arguments to pass to the plugin
            **kwargs  # Key word arguments to pass to the plugin
        ) -> Any:  # The result of the plugin execution
        "Execute a plugin's main functionality."
    
    def enable_plugin(
            self,
            plugin_name: str  # The name of the plugin to enable
        ) -> bool:  # True if plugin was enabled, False if not found
        "Enable a plugin."
    
    def disable_plugin(
            self,
            plugin_name: str  # The name of the plugin to disable
        ) -> bool:  # True if plugin was disabled, False if not found
        "Disable a plugin without unloading it."
```
